# Magics, Imports, and Versions

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [13]:
import collections
import csv
import functools
import json
import matplotlib.pyplot as plt
import numpy as np
import operator
import os
from pathlib import Path
import PIL
import pyedflib
import scipy
from scipy import signal
from subprocess import call
import sys
import typing
import urllib

In [14]:
print(f'__Python VERSION: {sys.version}')

try:
    print(f'__pyTorch VERSION: {torch.__version__}')
    PYTORCH = True
except: 
    print("Pytorch Not Installed")
    PYTORCH = False

try:
    print(f'__fastai VERSION: {fastai.__version__}')
except:
    print("fastai Not Installed")
    
print('__CUDA VERSION')

! nvcc --version

if PYTORCH:
    print(f'__CUDNN VERSION: {torch.backends.cudnn.version()}')
    print(f'__Number CUDA Devices: {torch.cuda.device_count()}')
    
print(f'__Devices')

try:
    call(["nvidia-smi", "--format=csv", "--query-gpu=index,name,driver_version,memory.total,memory.used,memory.free"])
    print(f'Active CUDA Device: GPU {torch.cuda.current_device()}')

    print (f'Available devices: {torch.cuda.device_count()}')
    print (f'Current cuda device {torch.cuda.current_device()}')
except:
    print("No GPUs Found")

__Python VERSION: 3.7.1 (default, Dec 14 2018, 19:28:38) 
[GCC 7.3.0]
Pytorch Not Installed
fastai Not Installed
__CUDA VERSION
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130
__Devices
No GPUs Found


# Path and Dirs

In [15]:
here = Path(f'{os.getcwd()}')
data_path = here.parent/'data'
raw_path = data_path/'raw'/'v1.5.0/edf'
with open(here.parent/'data_dict.json', 'r') as file:
     data_dict = json.load(file)
train_path = data_path/'augmented/train_val'
test_path = data_path/'augmented/test'

In [16]:
def ensure_dir(directory):
    if not os.path.exists(directory):
        print(f'Creating {directory}')
        os.makedirs(directory)

In [17]:
ensure_dir(data_path/'images/test')
img_path = data_path/'images/test'

In [18]:
# this is from 4 segments :0
len(list(test_path.iterdir()))

195620

# Math Functions

In [19]:
def divergence(F):
    """ compute the divergence of n-D scalar field `F` """
    """ https://stackoverflow.com/questions/11435809/compute-divergence-of-vector-field-using-python """
    return functools.reduce(np.add,np.gradient(F))

In [20]:
def scale(F):
    """ scale a n-D scalar field to [0-255] range for image creation """
    F *= (255.0/F.max())
    return F

In [21]:
def convolve_8NNavg(F):
    """ compute the average of the 8 nearest neighbors for each point in the n-D scalar field `F` """
    """ https://gis.stackexchange.com/questions/254753/calculate-the-average-of-neighbor-pixels-for-raster-edge """
    return scipy.ndimage.generic_filter(F, np.nanmean, size=3, mode='constant', cval=np.NaN)


# Test and Transform

In [22]:
gp = test_path/'le_00004151_s002_d2007_09_28_t003_ts003_ch041_h96_w8_o75_st000422.0_et000430.0_p00212_tp00222.npy'

In [23]:
for file in test_path.iterdir():
    # if file == gp:
    data = np.load(file)
    save_loc = img_path/f'{file.stem}.png'
    S = np.fft.fft(data)
    D = divergence(S)
    S = abs(S)
    D = abs(D)
    I = convolve_8NNavg(D)
    S = scale(S)
    D = scale(D)
    I = scale(I)
    img = np.stack([S,D,I], axis=2)
    img = PIL.Image.fromarray(img, 'RGB')
    img = img.resize((224, 224), PIL.Image.ANTIALIAS)
    img.save(save_loc)
    #plt.imshow(np.asarray(img))

KeyboardInterrupt: 